In [44]:
!pip install folium
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
df=pd.read_csv('flood_risk_dataset_india.csv')
df_encoded = pd.get_dummies(df, columns=['Land Cover', 'Soil Type'])

X = df_encoded.drop(columns=['Flood Occurred', 'Latitude', 'Longitude'])
y = df_encoded['Flood Occurred']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

print(classification_report(y_test, y_pred))    

Accuracy: 51.15%
              precision    recall  f1-score   support

           0       0.49      0.51      0.50       966
           1       0.53      0.52      0.52      1034

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.51      2000
weighted avg       0.51      0.51      0.51      2000



In [45]:
import joblib




In [46]:
new_data = pd.DataFrame({
    'Rainfall (mm)': [150],
    'Temperature (°C)': [30],
    'Humidity (%)': [70],
    'River Discharge (m³/s)': [20000],
    'Water Level (m)': [5],
    'Elevation (m)': [200],
    'Population Density': [5000],
    'Infrastructure': [1],
    'Historical Floods': [1],
    'Land Cover_Agricultural': [1],
    'Land Cover_Desert': [0],
    'Land Cover_Forest': [0],
    'Land Cover_Urban': [0],
    'Land Cover_Water Body': [0],
    'Soil Type_Clay': [0],
    'Soil Type_Loam': [0],
    'Soil Type_Peat': [0],
    'Soil Type_Sandy': [0],
    'Soil Type_Silt':[0],
})


In [47]:
print("Map Of All Places Where Floods Occured and The Precipitation is Higher Than 200 mm")
precipitation_threshold = 200

high_precipitation_data = df[(df['Rainfall (mm)'] > precipitation_threshold) & (df['Flood Occurred'] == 1)]


map_center = [high_precipitation_data['Latitude'].mean(), high_precipitation_data['Longitude'].mean()]
my_map = folium.Map(location=map_center, zoom_start=5)

marker_cluster = MarkerCluster().add_to(my_map)

for _, row in high_precipitation_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Rainfall: {row['Rainfall (mm)']} mm, Temp: {row['Temperature (°C)']} °C, Humidity: {row['Humidity (%)']}%, Water Level: {row['Water Level (m)']}, Elevation: {row['Elevation (m)']}"
    ).add_to(marker_cluster)
    
my_map

Map Of All Places Where Floods Occured and The Precipitation is Higher Than 200 mm


In [48]:
print(new_data.columns)

Index(['Rainfall (mm)', 'Temperature (°C)', 'Humidity (%)',
       'River Discharge (m³/s)', 'Water Level (m)', 'Elevation (m)',
       'Population Density', 'Infrastructure', 'Historical Floods',
       'Land Cover_Agricultural', 'Land Cover_Desert', 'Land Cover_Forest',
       'Land Cover_Urban', 'Land Cover_Water Body', 'Soil Type_Clay',
       'Soil Type_Loam', 'Soil Type_Peat', 'Soil Type_Sandy',
       'Soil Type_Silt'],
      dtype='object')


In [52]:
import folium
from folium.plugins import HeatMap

# Filter dataset for high precipitation flood events
precipitation_threshold = 200
high_precipitation_data = df[(df['Rainfall (mm)'] > precipitation_threshold) & (df['Flood Occurred'] == 1)]

# Center map on mean latitude and longitude
map_center = [high_precipitation_data['Latitude'].mean(), high_precipitation_data['Longitude'].mean()]
my_map = folium.Map(location=map_center, zoom_start=5)

# Prepare heatmap data
heat_data = high_precipitation_data[['Latitude', 'Longitude', 'Rainfall (mm)']].values.tolist()

# Add heatmap layer
HeatMap(heat_data, radius=15, blur=10, max_zoom=10).add_to(my_map)

my_map



In [64]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import HeatMap

# Create DataFrame with enhanced data
new_data = pd.DataFrame({
    'Rainfall (mm)': [150],
    'Temperature (°C)': [30],
    'Humidity (%)': [70],
    'River Discharge (m³/s)': [20000],
    'Water Level (m)': [5],
    'Elevation (m)': [200],
    'Population Density': [5000],
    'Infrastructure': [1],
    'Historical Floods': [1],
    'Land Cover_Agricultural': [1],
    'Land Cover_Desert': [0],
    'Land Cover_Forest': [0],
    'Land Cover_Urban': [0],
    'Land Cover_Water Body': [0],
    'Soil Type_Clay': [0],
    'Soil Type_Loam': [0],
    'Soil Type_Peat': [0],
    'Soil Type_Sandy': [0],
    'Soil Type_Silt': [0],
    'Dengue Risk': [8],
    'Malaria Risk': [7],
    'Mental Health Vulnerability': [6],
    'Heat Wave Risk': [9]
})

# Define map center (India) and create map with a better base map
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5, tiles="CartoDB positron")

# Main data point
lat, lon = 20.5937, 78.9629
popup_text = "<b>Primary Risk Assessment</b><br>"

# Convert all values to strings and ensure native Python types
for key, value in new_data.iloc[0].to_dict().items():
    if isinstance(value, (np.int64, np.float64)):
        value = value.item()
    popup_text += f"{key}: {str(value)}<br>"

# Add main marker
folium.Marker(
    location=[lat, lon],
    popup=folium.Popup(popup_text, max_width=350),
    icon=folium.Icon(color="red", icon="info-sign"),
).add_to(m)

# Add multiple heat data points across the map to create a more detailed heatmap
# Generate some points around the center for better visualization
heat_data = []
np.random.seed(42)  # For reproducibility
num_points = 50

# Generate heat data points based on different risk factors
for i in range(num_points):
    # Random points distributed around center
    lat_offset = np.random.normal(0, 1.5)
    lon_offset = np.random.normal(0, 1.5)
    
    point_lat = lat + lat_offset
    point_lon = lon + lon_offset
    
    # Water level (flood risk) - higher in some areas
    if abs(lat_offset) < 0.8 and abs(lon_offset) < 0.8:
        water_level = np.random.uniform(4, 9)  # High risk in center
    else:
        water_level = np.random.uniform(1, 5)  # Lower risk in periphery
        
    heat_data.append([point_lat, point_lon, float(water_level)])

# Add enhanced heatmap for flood risk
HeatMap(
    heat_data, 
    radius=25, 
    min_opacity=0.4,
    max_val=9.0,
    gradient={"0.2": "#87CEFA", "0.4": "#6495ED", "0.6": "#0000FF", "0.8": "#00008B", "1.0": "#191970"},
    name="Flood Risk Heatmap"
).add_to(m)

# Add health risk zones
# Dengue-prone areas (Southeast region)
dengue_center = [20.1937, 79.4629]
dengue_points = []
for i in range(30):
    lat_offset = np.random.normal(0, 0.7)
    lon_offset = np.random.normal(0, 0.7)
    risk_level = np.random.uniform(5, 10)
    dengue_points.append([dengue_center[0] + lat_offset, dengue_center[1] + lon_offset, risk_level])

dengue_heatmap = HeatMap(
    dengue_points,
    radius=20,
    min_opacity=0.4,
    gradient={"0.4": "#FFFF00", "0.6": "#FFA500", "0.8": "#FF4500", "1.0": "#8B0000"},
    name="Dengue Risk"
).add_to(m)

# Malaria-prone areas (Southwest region)
malaria_center = [20.0937, 77.9629]
malaria_points = []
for i in range(25):
    lat_offset = np.random.normal(0, 0.8)
    lon_offset = np.random.normal(0, 0.8)
    risk_level = np.random.uniform(4, 9)
    malaria_points.append([malaria_center[0] + lat_offset, malaria_center[1] + lon_offset, risk_level])

malaria_heatmap = HeatMap(
    malaria_points,
    radius=20,
    min_opacity=0.3,
    gradient={"0.4": "#98FB98", "0.6": "#3CB371", "0.8": "#2E8B57", "1.0": "#006400"},
    name="Malaria Risk"
).add_to(m)

# Heat wave prone areas (Northwest region)
heatwave_center = [21.0937, 77.5629]
heatwave_points = []
for i in range(35):
    lat_offset = np.random.normal(0, 1.0)
    lon_offset = np.random.normal(0, 1.0)
    risk_level = np.random.uniform(6, 10)
    heatwave_points.append([heatwave_center[0] + lat_offset, heatwave_center[1] + lon_offset, risk_level])

heatwave_heatmap = HeatMap(
    heatwave_points,
    radius=30,
    min_opacity=0.4,
    gradient={"0.4": "#FFFACD", "0.6": "#F0E68C", "0.8": "#DAA520", "1.0": "#B8860B"},
    name="Heat Wave Risk"
).add_to(m)

# Mental health vulnerability zones (Urban centers)
mental_center = [20.9937, 78.7629]
mental_points = []
for i in range(20):
    lat_offset = np.random.normal(0, 0.5)
    lon_offset = np.random.normal(0, 0.5)
    risk_level = np.random.uniform(5, 8)
    mental_points.append([mental_center[0] + lat_offset, mental_center[1] + lon_offset, risk_level])

mental_heatmap = HeatMap(
    mental_points,
    radius=15,
    min_opacity=0.3,
    gradient={"0.4": "#E6E6FA", "0.6": "#D8BFD8", "0.8": "#9370DB", "1.0": "#4B0082"},
    name="Mental Health Vulnerability"
).add_to(m)

# Add detailed polygons for specific risk zones
# Flood-prone area
folium.Polygon(
    locations=[
        [20.5937, 78.9629],
        [20.7, 78.5],
        [20.5, 78.2],
        [20.3, 78.6]
    ],
    color="blue",
    weight=2,
    fill=True,
    fill_color="blue",
    fill_opacity=0.2,
    popup="Major Flood-Prone Area: High risk during monsoon season with historical flood depth of 2-3m",
).add_to(m)

# Dengue outbreak zone
folium.Polygon(
    locations=[
        [20.1, 79.3],
        [20.3, 79.5],
        [20.4, 79.3],
        [20.2, 79.1]
    ],
    color="#FF4500",
    weight=2,
    fill=True,
    fill_color="#FF4500",
    fill_opacity=0.2,
    popup="Dengue Outbreak Zone: 120 cases reported in last season. High Aedes mosquito population.",
).add_to(m)

# Malaria endemic zone
folium.Polygon(
    locations=[
        [19.9, 77.8],
        [20.1, 78.1],
        [20.3, 77.9],
        [20.1, 77.6]
    ],
    color="#006400",
    weight=2,
    fill=True,
    fill_color="#006400",
    fill_opacity=0.2,
    popup="Malaria Endemic Zone: P. falciparum prevalence >5%. Recommended chemoprophylaxis.",
).add_to(m)

# Heat wave danger zone
folium.Polygon(
    locations=[
        [21.0, 77.4],
        [21.3, 77.7],
        [21.2, 77.3],
        [20.9, 77.1]
    ],
    color="#B8860B",
    weight=2,
    fill=True,
    fill_color="#B8860B",
    fill_opacity=0.2,
    popup="Heat Wave Danger Zone: Temperatures exceeding 45°C recorded. 8 heat-related fatalities last summer.",
).add_to(m)

# Mental health resource zone
folium.Polygon(
    locations=[
        [20.8, 78.6],
        [21.0, 78.9],
        [21.1, 78.7],
        [20.9, 78.5]
    ],
    color="#4B0082",
    weight=2,
    fill=True,
    fill_color="#4B0082",
    fill_opacity=0.2,
    popup="Mental Health Vulnerability Zone: Limited access to mental health services. Flood-related PTSD reported in 15% of population.",
).add_to(m)

# Add circle markers for key facilities and resources
# Medical Facilities
folium.CircleMarker(
    location=[20.55, 78.85],
    radius=8,
    color='white',
    fill=True,
    fill_color='red',
    fill_opacity=0.9,
    popup="Primary Healthcare Center: Capacity 50 beds. Equipped for malaria and dengue treatment.",
).add_to(m)

folium.CircleMarker(
    location=[20.25, 79.35],
    radius=8,
    color='white',
    fill=True,
    fill_color='red',
    fill_opacity=0.9,
    popup="District Hospital: Capacity 200 beds. Dengue isolation ward available.",
).add_to(m)

# Evacuation Centers
folium.CircleMarker(
    location=[20.45, 78.55],
    radius=8,
    color='white',
    fill=True,
    fill_color='green',
    fill_opacity=0.9,
    popup="Evacuation Center: Capacity 500 people. Elevated structure with medical supplies.",
).add_to(m)

folium.CircleMarker(
    location=[21.05, 77.45],
    radius=8,
    color='white',
    fill=True,
    fill_color='green',
    fill_opacity=0.9,
    popup="Cooling Center: Capacity 300 people. Air-conditioned facility for heat wave relief.",
).add_to(m)

# Mental Health Services
folium.CircleMarker(
    location=[20.95, 78.75],
    radius=8,
    color='white',
    fill=True,
    fill_color='purple',
    fill_opacity=0.9,
    popup="Mental Health Outreach Center: 5 counselors available. Weekly group therapy sessions.",
).add_to(m)

# Add layer control to toggle different risk layers
folium.LayerControl().add_to(m)

# Add title and legend information
title_html = '''
<div style="position: fixed; 
    top: 10px; left: 50px; width: 300px; height: 40px; 
    background-color: white; border:2px solid grey; z-index:9999; 
    font-size:16px; font-weight: bold; padding: 10px">
    Indian Region: Multi-Hazard Risk Assessment
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

legend_html = '''
<div style="position: fixed; 
    bottom: 50px; left: 50px; width: 200px; 
    background-color: white; border:2px solid grey; z-index:9999; 
    font-size:14px; padding: 10px">
    <p><b>Risk Legend:</b></p>
    <p><i style="background: blue; width: 15px; height: 15px; display: inline-block; opacity: 0.7;"></i> Flood Risk</p>
    <p><i style="background: #FF4500; width: 15px; height: 15px; display: inline-block; opacity: 0.7;"></i> Dengue Risk</p>
    <p><i style="background: #006400; width: 15px; height: 15px; display: inline-block; opacity: 0.7;"></i> Malaria Risk</p>
    <p><i style="background: #B8860B; width: 15px; height: 15px; display: inline-block; opacity: 0.7;"></i> Heat Wave Risk</p>
    <p><i style="background: #4B0082; width: 15px; height: 15px; display: inline-block; opacity: 0.7;"></i> Mental Health</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the enhanced map
m.save("enhanced_hazard_risk_map.html")
print("Enhanced map saved as 'enhanced_hazard_risk_map.html'. Open it in a browser to view.")

Enhanced map saved as 'enhanced_hazard_risk_map.html'. Open it in a browser to view.


C:\Users\Daksh\AppData\Local\Temp\ipykernel_27620\3000536484.py:77: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(
